# Programming Assignment 2: Naive Bayes
## Part 1: Language Modelling and Text Generation

#### Name: Muhammad Waleed
#### Roll Number: 22030017

### Instructions
*   In this part of the assignment you will be implementing an n-gram model for text-generation.
*   Your code must be in the Python programming language.
*   You are encouraged to use procedural programming and throughly comment your code.
*   For Part 1, in addition to standard libraries i.e. numpy, pandas, regex, matplotlib and scipy, you can use [UrduHack](https://docs.urduhack.com/en/stable/index.html) for tokenization, and [NLKT](https://www.nltk.org/) for training your n-grams. However, no other machine learning toolkits or libraries are allowed.
*   **Carefully read the submission guidelines, plagiarism and late days policy.**

### Submission Guidelines
Submit your code both as notebook file (.ipynb) and python script (.py) as individual files on LMS. Name both files as RollNumber_PA2_PartNum, i.e. this part should be named as `2xxxxxxx_PA4_1`. If you don’t know how to save .ipynb as .py see [this](https://i.stack.imgur.com/L1rQH.png). Failing to submit any one of them might result in the reduction of marks. All cells **MUST** be run to get credit.

### Plagiarism Policy
The code **MUST** be done independently. Any plagiarism or cheating of work from others or the internet will be immediately referred to the DC. If you are confused about what constitutes plagiarism, it is **YOUR** responsibility to consult with the instructor or the TA in a timely manner. No “after the fact” negotiations will be possible. The only way to guarantee that you do not lose marks is **DO NOT LOOK AT ANYONE ELSE'S CODE NOR DISCUSS IT WITH THEM**.

### Late Days Policy

The deadline for the assignment is final. However, in order to accommodate all the 11th-
hour issues, there is a late submission policy i.e. you can submit your assignment within
3 days after the deadline with a 25% deduction each day.


### Introduction
An n-gram is a contiguous sequence of n words. For example "Machine" is a unigram, "Machine Learning" is a bigram and "Machine Learning PA2" is a trigram. In language modeling, n-gram models are probabilistic models of text that use word dependencies and context to predict the likelihood of occurence of an n-gram, i.e. predicting the nth word in an n-gram based on the previous n-1 words:
$$
P(ngram) =  P(word|context) = P(x^{n}|x^{n-1},...,x^{1})
$$
One use of the predictions made by such a model is text generation. In this part you will be training your own n-gram model and using it to generate text after learning from the provided Urdu short stories. 
<br><br>
For additional details of the working of n-gram models, you can also consult [Chapter 3](https://web.stanford.edu/~jurafsky/slp3/3.pdf) of the Speech and Language Processing book as and references.


### Dataset
You will be using the Urdu short stories by Patras Bukhari given in the folder `Urdu Short Stories` in the PA2 zip file for the purposes of this part of the assignment. This contains 6 stories of varying lengths which will serve as inputs for your n-gram model. 
You're required to implement an n-gram model that uses the given stories to generate Urdu text that mimics the input stories.

Start by importing all required libraries here.

In [ ]:
# using google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install urduhack

In [ ]:
# import all required libraries here
import numpy as np
import os
import random
from urduhack.tokenization import sentence_tokenizer
from urduhack.tokenization import word_tokenizer
from urduhack.preprocessing import normalize_whitespace, remove_punctuation
import urduhack
from nltk import ngrams
urduhack.download()

89078/89078 [==============================] - 0s 0us/step


### 1.1 - Loading and Preprocessing the Dataset

Read in the short story files given and tokenize the text to be preprocessed.

In [ ]:
# code here
DataP1 = "/content/drive/MyDrive/Assignment 4/Assignment 4/DataP1"
DataP2 = "/content/drive/MyDrive/Assignment 4/Assignment 4/DataP2"

# reading and saving the content of each file
all_data = []
for i in os.listdir(DataP1):
  if i.endswith(".txt"):
    print(i)
    txt = DataP1 + "/" + i
    with open(txt, "r") as f:
      lines = f.read()
      all_data.append(lines)

hostel mein parhna.txt
maibal aur main.txt
cinema ka ishq.txt
sawere.txt
lahore ka jughrafiya.txt
dost k naam.txt


Preprocess the tokenized data. Go through the data and use your own discretion to decide on what kind of pre-processing might be required.

In [ ]:
# code here
# Generating tokens and and applying pre-procesing 
all_tokens = []
for i in all_data:
  normalized_text = remove_punctuation(all_data[0])
  sentences = sentence_tokenizer(normalized_text)
  print(len(sentences))
  for j in sentences:
    tokens = word_tokenizer(j)
    all_tokens += tokens

128
12/12 [==============================] - 2s 156ms/step
128
12/12 [==============================] - 2s 167ms/step
128
12/12 [==============================] - 2s 157ms/step
128
12/12 [==============================] - 2s 153ms/step
128
12/12 [==============================] - 2s 158ms/step
128
12/12 [==============================] - 2s 155ms/step


In [ ]:
# Saving tokens so do not need to generate again and again
import pickle
with open('all_tokens.pkl', 'wb') as f:
  pickle.dump(all_tokens, f)

In [ ]:
# Using the save tokens
file = open("/content/drive/MyDrive/Assignment 4/all_tokens.pkl", 'rb')
all_tokens = pickle.load(file)

### 1.2 - Creating Unigrams

Start by training a unigram model. For a unigram model, the n-gram probability is approximated by probability of the word in the unigram, as the model assumes independence:

$$
P(word) = \frac{n}{N}
$$

where n = count of the word in the corpus and N = total number of words in the corpus.

Generate a list of unigrams. Print the first 10 unigrams obtained.

In [ ]:
# Generating unigrams
all_uni_grams = ngrams(all_tokens, 1)
uni_grams = []

for i in all_uni_grams:
 uni_grams.append(i[0])

print(uni_grams[0:10])

['ہم', 'نے', 'کالج', 'میں', 'تعلیم', 'تو', 'ضرور', 'پائی', 'اور', 'رفتہ']


Find the probabilities for each unique unigram. 

In [ ]:
# Finding probabilities

prob_uni_grams = [] # probabilities 
uni_visited = []  # distinct uni_grams
n = len(uni_grams) 
for i in uni_grams:
  if i not in uni_visited:
    prob_uni_grams.append(uni_grams.count(i)/n)
    uni_visited.append(i)
print(f"Probability of {uni_visited[0]} is {prob_uni_grams[0]}")

Probability of ہم is 0.012141022647676861


### 1.3 - Creating Bigrams
Now train a bigram model. 

Generate a list of bigrams. Print the first 10 bigrams obtained.

In [ ]:
# Generating biigrams

all_bi_grams = ngrams(all_tokens, 2)
bi_grams = []

for i in all_bi_grams:
 bi_grams.append(i)

print(bi_grams[0:10])

[('ہم', 'نے'), ('نے', 'کالج'), ('کالج', 'میں'), ('میں', 'تعلیم'), ('تعلیم', 'تو'), ('تو', 'ضرور'), ('ضرور', 'پائی'), ('پائی', 'اور'), ('اور', 'رفتہ'), ('رفتہ', 'رفتہ')]


Find the probabilities for each unique bigram. 

In [ ]:
# Finding probabilities

prob_bi_grams = [] # probabilities 
bi_visited = [] # distinct bi_grams
for i in bi_grams:
  if i not in bi_visited:
    n = uni_grams.count(i[0])
    prob_bi_grams.append(bi_grams.count(i)/n)
    bi_visited.append(i)
print(f"Probability of {bi_visited[1]} is {prob_bi_grams[1]}")

Probability of ('نے', 'کالج') is 0.01818181818181818


### 1.4 - Creating Trigrams
Lastly train a trigram model.

Generate a list of trigrams. Print the first 10 trigrams obtained.

In [ ]:
# Generating trigrams

all_tri_grams = ngrams(all_tokens, 3)
tri_grams = []

for i in all_tri_grams:
 tri_grams.append(i)

print(tri_grams[0:10])

[('ہم', 'نے', 'کالج'), ('نے', 'کالج', 'میں'), ('کالج', 'میں', 'تعلیم'), ('میں', 'تعلیم', 'تو'), ('تعلیم', 'تو', 'ضرور'), ('تو', 'ضرور', 'پائی'), ('ضرور', 'پائی', 'اور'), ('پائی', 'اور', 'رفتہ'), ('اور', 'رفتہ', 'رفتہ'), ('رفتہ', 'رفتہ', 'بی')]


Find the probabilities for each unique trigram. 

In [ ]:
# Finding probabilities

prob_tri_grams = [] # probabilities 
tri_visited = [] # distinct tri_grams
for i in tri_grams:
  if i not in tri_visited:
    n = bi_grams.count(i[0:2])
    prob_tri_grams.append(tri_grams.count(i)/n)
    tri_visited.append(i)
print(f"Probability of {tri_visited[1]} is {prob_tri_grams[1]}")

Probability of ('نے', 'کالج', 'میں') is 1.0


### 1.5 - Generating Text
Generate a paragraph with ten sentences each containing 9-15 words (pick the length of the sentence randomly within this range) using you language model. Start with trigrams, use back-off technique (i.e. use n-1 gram) if a token is not available. 

For each word prediction, get top 5 most probabale words using the n-gram model and then pick the next word randomly from within these. This is being done to avoid excessive repetitive sequences in your generated text.

In [ ]:
prob = prob_tri_grams.copy()
visited = tri_visited.copy()

def random_max(prob, visited): # method to get top 5 most probabale words
  max_prob = []
  max_visit = []
  for i in range(0, 5):
    m = max(prob)
    v = visited[prob.index(m)]
    max_prob.append(m)
    max_visit.append(v)

  choice = random.randrange(5)
  choose = visited[choice]
  prob.remove(max_prob[choice])
  for i in visited:
    if choose == i:
      visited.remove(i)
  return choose, prob, visited

def find_max_prob(find_prob_words, visited, prob):  # finding maximum probability
  max_prob = []
  for i in find_prob_words:
    max_prob.append(prob[visited.index(i)])
  choose = max_prob.index(max(max_prob))
  prob.remove(prob[visited.index(find_prob_words[choose])])
  visited.remove(find_prob_words[choose])
  return find_prob_words[choose], visited, prob
  
# method to implement n-1 grams
def second_random(prob_tri_grams, prob_bi_grams, prob_uni_grams, tri_visited, bi_visited, uni_visited, words):
  rand_word, prob1, tri_visited = random_max(prob_tri_grams, tri_visited)
  sentence = ""
  for i in range(words):
    first_word = rand_word[0]
    second_word = rand_word[1]
    third_word = rand_word[2]

    uni_prefix = []
    tri_prefix = []
    bi_prefix = []
    for j in tri_visited:
      if first_word != j[0] and second_word == j[0] and third_word == j[1]: # if token is present in trigrams then use trigrams
        tri_prefix.append(j)

    if tri_prefix == [] and bi_prefix != []: # if token is not present in trigrams then use bigrams
      for j in bi_visited:
        if third_word == j[0]:
          bi_prefix.append(j)
      result, bi_visited, prob_bi_grams = find_max_prob(bi_prefix, bi_visited, prob_bi_grams)
      rand_word = (result[0], result[1])

    elif tri_prefix != []:
      result, tri_visited, prob_tri_grams = find_max_prob(tri_prefix, tri_visited, prob_tri_grams)
      rand_word = (result[0], result[1], result[2])

    elif tri_prefix == [] and bi_prefix == []: # if token is not present in trigrams and bigrams then use unigrams
      ind = prob_uni_grams.index(max(prob_uni_grams))
      result = uni_visited[ind]
      prob_uni_grams.pop(ind)
      uni_visited.pop(ind)
      rand_word = (rand_word[-2], rand_word[-1], result)
    if i < 1 and tri_prefix != []:
      sentence += " " + first_word + " " + result[0] + " " + result[1] + " " + result[2]
    
    elif i < 1 and bi_prefix != []:
      sentence += " " + first_word + " " + result[0] + " " + result[1]

    elif len(result[-1]) == 1:
      try:
        sentence += " " + result
      except:
        sentence += " " + result[-1]

    else:
      sentence += " " + result[-1]

  return sentence + "۔", prob_tri_grams, prob_bi_grams, prob_uni_grams, tri_visited, bi_visited, uni_visited

sentence, prob_tri_grams, prob_bi_grams, prob_uni_grams, tri_visited, bi_visited, uni_visited = second_random(prob_tri_grams, prob_bi_grams, prob_uni_grams, tri_visited, bi_visited, uni_visited, words=9)
print(sentence)

 کے گھر اس سے اگلے سال یہ دلیل پیش کی جا سکے۔


In [ ]:
# Generating 10 sentences randomly
for i in range(10):
  sentence, prob_tri_grams, prob_bi_grams, prob_uni_grams, tri_visited, bi_visited, uni_visited = second_random(prob_tri_grams, prob_bi_grams, prob_uni_grams, tri_visited, bi_visited, uni_visited, words=9)
  print(sentence)

 کا جو ایک علم دوست خاندان سے تعلق رکھتا ہو لوگوں کے۔
 ماموں کے گھر میں زندگی ہے تو خزاں کے بھی گزر جائیں۔
 گرافی تالیف دندان عینک ایجنٹوں غرض بالانشیں پیشے سیکھے۔
 پھلنے پھولنے کا موقع نہ ملے گا اور تعلیم کا اصلی مقصد۔
 اصلی مقصد فوت ہو جائے تاکہ وقت پر ان ہیںصدمہ نہ ہو۔
 ہو جائے گا بس یہی ایک کسر باقی رہ گئی ہے اس۔
 تھوڑے فن مولا گردونواح شہر پبلک قتعانا تھیاس تحصیلدار۔
 وہی ہوا جس کا ہمیں خوف تھا ہم روز بروز بڑھتی گئی۔
 سیجم نے لگی سنیما جانے کہ اجازت کبھی کبھار مل جاتی تھی۔
 کر سکتا تھا تھیٹر کے معاملے میں ہماری معلومات اندر سبھا سے۔


### 1.6 - Discussion and Evaluation

- Analyze the text generated, and mention 3 distinct observations. Also compare it with the input text and how different it is and why might that be.
- Is going upto n=3 enough? What do you think would be a good value of n and why?

Answer here:
- Observations:

1) The start of the each sentence makes more sense than middle of sentence and middle of sentence makes more sense than the end.

2) We can clearly see that as during start of the sentence the trigrams are used, then if trigrams token not found than bigrams and in the last unigrams.

3) Trigrams are making more senese than bigrams and bigrams are making more sense than unigrams.

- The n upto 4 should be enough as when n increases furthure the input feature become large and thus it will not work well